Install Requirements

In [3]:
!pip install -r requirements.txt

Open CSV

In [2]:
import csv

# Ruta del archivo CSV
archivo_csv = 'data/resume.csv'

# Lista para almacenar los datos del CSV
datos = []

# Abrir el archivo CSV y leer los datos
with open(archivo_csv, newline='') as csvfile:
    lector_csv = csv.reader(csvfile, delimiter=',')
    # Iterar sobre cada fila del CSV
    for fila in lector_csv:
        datos.append(fila)

In [3]:
len(datos)

2485

In [4]:
def chunk_by_words(text, chunk_size=300, overlap=10):
    tokens = text.split()
    chunks = []
    i = 0
    while i < len(tokens):
        chunks.append(" ".join(tokens[i:i+chunk_size]))
        i += chunk_size - overlap
    return chunks

In [5]:
speech = "Nosotros elegimos ir a la luna. Elegimos ir a la luna. Elegimos ir a la luna en esta década y hacer otras cosas no porque sean fáciles, sino porque son difíciles. Porque esa meta servirá para organizar y medir lo mejor de nuestras energías y habilidades, porque ese desafío es uno que estamos dispuestos a aceptar."

chunks = chunk_by_words(speech, chunk_size=20, overlap=5)
for idx, chunk in enumerate(chunks):
    print(f"Chunk {idx}: {chunk}\n")
    print(f"Chunk {idx}: {chunk}\n")

Chunk 0: Nosotros elegimos ir a la luna. Elegimos ir a la luna. Elegimos ir a la luna en esta década y

Chunk 0: Nosotros elegimos ir a la luna. Elegimos ir a la luna. Elegimos ir a la luna en esta década y

Chunk 1: luna en esta década y hacer otras cosas no porque sean fáciles, sino porque son difíciles. Porque esa meta servirá

Chunk 1: luna en esta década y hacer otras cosas no porque sean fáciles, sino porque son difíciles. Porque esa meta servirá

Chunk 2: difíciles. Porque esa meta servirá para organizar y medir lo mejor de nuestras energías y habilidades, porque ese desafío es

Chunk 2: difíciles. Porque esa meta servirá para organizar y medir lo mejor de nuestras energías y habilidades, porque ese desafío es

Chunk 3: habilidades, porque ese desafío es uno que estamos dispuestos a aceptar.

Chunk 3: habilidades, porque ese desafío es uno que estamos dispuestos a aceptar.



In [107]:
chunked_documents = []
chunk_unique_id = 0
for doc_id, document in enumerate(datos):
    chunks = chunk_by_words(document[1])
    for chunk_id, chunk in enumerate(chunks):
        chunked_documents.append({
            "id": chunk_unique_id,
            "document_id": chunk_id,
            "person_id": document[0],
            "text": chunk
        })
        chunk_unique_id += 1

In [108]:
print(len(chunked_documents))

8218


In [109]:
chunked_documents[1]

{'id': 1,
 'document_id': 0,
 'person_id': '16852973',
 'text': "HR ADMINISTRATOR/MARKETING ASSOCIATE HR ADMINISTRATOR Summary Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management. Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service. Highlights Focused on customer satisfaction Team management Marketing savvy Conflict resolution techniques Training and development Skilled multi-tasker Client relations specialist Accomplishments Missouri DOT Supervisor Training Certification Certified by IHG in Customer Loyalty and Marketing by Segment Hilton Worldwide General Manager Training Certification Accomplished Trainer for cross server hospitality systems such as Hilton OnQ , Micros Opera PMS , Fidelio OPERA Reservation System (ORS) , Holidex Completed courses and seminars in customer service, sales strategies, inventory control, loss prevention, safety, time man

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

/Users/gancoce/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [75]:
def get_embedding(text):
    return model.encode(text)

In [11]:
from annoy import AnnoyIndex
import os

embedding_size = 768
index_name = "full-index-mananeras.ann"

In [110]:
chunk_id_mapping = {}
for chunk in chunked_documents:
    chunk_id_mapping[chunk["id"]] = chunk

In [111]:
print(chunk_id_mapping[1])

{'id': 1, 'document_id': 0, 'person_id': '16852973', 'text': "HR ADMINISTRATOR/MARKETING ASSOCIATE HR ADMINISTRATOR Summary Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management. Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service. Highlights Focused on customer satisfaction Team management Marketing savvy Conflict resolution techniques Training and development Skilled multi-tasker Client relations specialist Accomplishments Missouri DOT Supervisor Training Certification Certified by IHG in Customer Loyalty and Marketing by Segment Hilton Worldwide General Manager Training Certification Accomplished Trainer for cross server hospitality systems such as Hilton OnQ , Micros Opera PMS , Fidelio OPERA Reservation System (ORS) , Holidex Completed courses and seminars in customer service, sales strategies, inventory control, loss prevention, safety, time manage

In [112]:
if not os.path.exists(index_name):
    index = AnnoyIndex(embedding_size, 'angular')

    for idx, chunk in chunk_id_mapping.items():

        v = get_embedding("person_id: " + chunk["resume_id"]+", resume: "+chunk["text"])

        index.add_item(idx, v)

    index.build(10)
    index.save(index_name)

In [113]:
index = AnnoyIndex(embedding_size, 'angular')
index.load(index_name)

True

In [114]:
pregunta = "¿Give me top 3 resumes and person_id that fits the following requirement?: Senior Software Developer  Responsibilities:  Develop web applications using Java and React. Design and maintain robust APIs. Collaborate on technical design and software architecture. Optimize performance and resolve technical issues. Document code and processes. Requirements:  3+ years of software development experience. Proficiency in Java and React. Experience with RESTful APIs. Strong analytical and problem-solving skills. Communication skills in Spanish/English. Desirable:  Experience with agile methodologies. Knowledge of databases and Git."

In [115]:
print(pregunta)
embedding_pregunta = get_embedding(pregunta)
#print(embedding_pregunta)

¿Give me top 3 resumes and person_id that fits the following requirement?: Senior Software Developer  Responsibilities:  Develop web applications using Java and React. Design and maintain robust APIs. Collaborate on technical design and software architecture. Optimize performance and resolve technical issues. Document code and processes. Requirements:  3+ years of software development experience. Proficiency in Java and React. Experience with RESTful APIs. Strong analytical and problem-solving skills. Communication skills in Spanish/English. Desirable:  Experience with agile methodologies. Knowledge of databases and Git.


In [123]:
ids_potenciales_respuestas = index.get_nns_by_vector(embedding_pregunta, 5)
ids_potenciales_respuestas

[7233, 6837, 441, 3876, 6346]

In [124]:
potenciales_respuestas = [chunk_id_mapping[idx] for idx in ids_potenciales_respuestas]
print(potenciales_respuestas)
texto_potencial = [f"{chunk['person_id']}: {chunk['text']}" for chunk in potenciales_respuestas]
texto_potencial

[{'id': 7233, 'document_id': 0, 'person_id': '39247950', 'text': "SOFTWARE ENGINEER Qualifications C# 3.0, PL/SQL, JavaScript, HTML 4, CSS 2 Framework: .NET 3.5 Database: SQL Server 2008, Oracle 9i Operating systems: Windows 98/XP, Windows server 2000/2005, UNIX Tools/Services: TOAD, HP Service Manager, WinSCP, PuTTY, PM Smart, vi text editor, Visual SourceSafe and BusinessObjects XI 3.1 Universe Designer, Desktop Intelligence, Central Management Console Domain: Banking Work Experience Software Engineer July 2010 to March 2014 Company Name 3 years and 9 months of experience in the development, support and enhancement of web and windows applications and in the implementation of cost effective valueadds Designed and developed webbased tools in ASP .NET 3.5, using the programming and C# PL/SQL procedural languages namely and Enhanced EBusiness applications and managed technological issues by analyzing codes and providing customizable solutions using SQL and Business Objects XI 3.1 tools A

["39247950: SOFTWARE ENGINEER Qualifications C# 3.0, PL/SQL, JavaScript, HTML 4, CSS 2 Framework: .NET 3.5 Database: SQL Server 2008, Oracle 9i Operating systems: Windows 98/XP, Windows server 2000/2005, UNIX Tools/Services: TOAD, HP Service Manager, WinSCP, PuTTY, PM Smart, vi text editor, Visual SourceSafe and BusinessObjects XI 3.1 Universe Designer, Desktop Intelligence, Central Management Console Domain: Banking Work Experience Software Engineer July 2010 to March 2014 Company Name 3 years and 9 months of experience in the development, support and enhancement of web and windows applications and in the implementation of cost effective valueadds Designed and developed webbased tools in ASP .NET 3.5, using the programming and C# PL/SQL procedural languages namely and Enhanced EBusiness applications and managed technological issues by analyzing codes and providing customizable solutions using SQL and Business Objects XI 3.1 tools Awarded for outstanding performance and ontime delivery

Create OpenAI method to get answers

In [19]:
from openai import OpenAI

client = OpenAI()

def query_llm(prompt, model="gpt-3.5-turbo"):
    completions = client.chat.completions.create(

    model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
    )

    return completions.choices[0].message.content

In [20]:
def get_answer(question, context_documents):
    context_divider = "\n---\n"
    question = question.strip()

    context = context_divider.join(context_documents)

    template_rag = """You are an assistant in the Human Resources department of an outsourcing company.
Respond to the question based on the context required by the employee.
The context is delimited by the inverted commas.

```
{context}
```

Question: {question}
"""
    final_prompt = template_rag.format(context=context, question=question)
    return query_llm(final_prompt)

In [125]:
print(pregunta)
print()
print(get_answer(pregunta, texto_potencial))

¿Give me top 3 resumes and person_id that fits the following requirement?: Senior Software Developer  Responsibilities:  Develop web applications using Java and React. Design and maintain robust APIs. Collaborate on technical design and software architecture. Optimize performance and resolve technical issues. Document code and processes. Requirements:  3+ years of software development experience. Proficiency in Java and React. Experience with RESTful APIs. Strong analytical and problem-solving skills. Communication skills in Spanish/English. Desirable:  Experience with agile methodologies. Knowledge of databases and Git.

Based on the provided resumes, the top 3 candidates that fit the requirements for a Senior Software Developer position are:

1. Person ID: 12415691
   - Title: Information Designer
   - Qualifications: Proficiency in Java, experience with RESTful APIs, strong problem-solving skills, communication skills in English.
   - Experience: 3+ years of professional experience 